In [0]:
%pip install gspread oauth2client

In [0]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define the scope and credentials for Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/Workspace/Users/mhemanthkmr143@gmail.com/service.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet = client.open("Attendance  (Responses)").sheet1

# Get all records from the sheet
data = sheet.get_all_records()

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, to_date

df = spark.createDataFrame(data)

# Rename columns to snake case
df = df.withColumnRenamed("Timestamp", "timestamp") \
       .withColumnRenamed("Email Address", "email_address") \
       .withColumnRenamed("Name", "name") \
       .withColumnRenamed("Batch", "batch") \
       .withColumnRenamed("Student ID", "student_id")

# Convert string to timestamp
df = df.withColumn("timestamp", to_date("timestamp", "M/d/yyyy H:mm:ss"))

# Add a unique identifier column starting from 1 to n
df = df.withColumn("id", monotonically_increasing_id() + 1)

# Reorder columns to display id as first
columns = ["id"] + [col for col in df.columns if col != "id"]
df = df.select(columns)

display(df)

In [0]:
from pyspark.sql.functions import current_date

# Add a load_date column to the new data
df = df.withColumn("load_date", current_date())

# Write the merged data back to Unity Catalog
df.write.format("delta").mode("overwrite").saveAsTable("besant.attendance.student_attendance")